In [ ]:
#######################################################################################################################
# Project: Deep Virtual Rapport Agent (data preprocessing)
#
#     Jan Ondras (jo951030@gmail.com)
#     Institute for Creative Technologies, University of Southern California
#     April-October 2019
#
#######################################################################################################################
# Annotate OpenFace features from the hatice2010 dataset with ground-true head gestures.
#
#     Each frame will be annotated with ground-truth annotations of nod and shake head gestures
#
#     Input features: dvra_datasets/hatice2010/openface_features 
#     Output features: dvra_datasets/hatice2010/annotated_features
#######################################################################################################################

In [1]:
###########################################################
import numpy as np
random_seed = 37
np.random.seed(random_seed)
from tensorflow import set_random_seed
set_random_seed(random_seed)
###########################################################

# For each recording
#     Resample feature dataframe
#     Add first derivatives of selected features
#     Annotate frames
#     Save as new annotated dataframe, 0 => not a nod, 1 => nod
#     Add subject ID and head gesture class into the new filename

import os
import glob
import pandas as pd
import scipy.signal
from scipy import interpolate
import time
import pympi    # Import pympi to work with elan files

# Unified frame rate
FRAME_RATE = 30.

# Features whose first and second derivatives will be calculated
diff_selected_features = [
    ' pose_Tx', 
    ' pose_Ty', 
    ' pose_Tz', 

    ' pose_Rx', 
    ' pose_Ry', 
    ' pose_Rz',

    ' p_rx', 
    ' p_ry',
    ' p_rz'
    
    # add landmarks?
]

# Map session id prefix to subject ID
sessid_prefix_to_sid = {
    # Classes nod and shake
    '2008.12.05.16.03.15_Operator': 1, 
    '2008.12.05.16.03.15_User': 2, 
    
    '2008.12.14.14.47.07_Operator': 1, 
    '2008.12.14.14.47.07_User': 2, 
    
    '2008.12.19.11.03.11_Operator': 1, 
    '2008.12.19.11.03.11_User': 3,
    
    '2009.01.06.12.41.42_Operator': 1, 
    
    '2009.01.06.14.53.49_Operator': 1, 
    '2009.01.06.14.53.49_User': 4, 
    
    '2009.01.28.15.35.20_Operator': 5, 
    '2009.01.28.15.35.20_User': 1, 
    
    '2009.01.30.12.00.35_Operator': 5, 
    '2009.01.30.12.00.35_User': 2, 
    
    '2009.05.22.15.17.45_User': 6,
    
    '2009.05.25.11.23.09_User': 7, 
    
    '2009.05.26.10.19.53_User': 8, 
    
    # Class other
    '2008': 9, 
    'Alex': 10, 
    'Alis': 11, 
    'anto': 12, 
    'Liam': 13, 
    'maja': 14, 
    'rodA': 15
}

input_features_dir = '/home/ICT2000/jondras/dvra_datasets/hatice2010/openface_features'
output_dir = '/home/ICT2000/jondras/dvra_datasets/hatice2010/annotated_features'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
start_time = time.time()    
cnt = 0
for feature_file in sorted(glob.glob(input_features_dir + '/*/*.csv')):
    
    head_gesture = feature_file.split('/')[-2]
    sessid = feature_file.split('/')[-1][:-4]
    if head_gesture == 'other':
        sessid_prefix = sessid[:4]
    else:
        sessid_prefix = '_'.join(sessid.split('_')[:2])
    sid = sessid_prefix_to_sid[sessid_prefix]
    
    print(f'Processing SESSID {sid}\n\t head gesture: {head_gesture}\t sessid_prefix: {sessid_prefix}\t sid: {sid}')
   
    org_df = pd.read_csv(feature_file)
    print(f'\t original dataframe length {len(org_df)}')
#     print(len(org_df), org_df)
    
    # Resample feature dataframe to common frame rate, if needed
    csv_frame_rate = (org_df.shape[0] - 1) / np.sum(np.diff(org_df[' timestamp']))
    print(f'\tcsv frame rate: {csv_frame_rate}')
    if round(csv_frame_rate) == FRAME_RATE:
        print(f'\tNOT resampling')
        new_df = org_df.copy()
    else:
        print(f'\tREsampling from {csv_frame_rate} to {FRAME_RATE}')
        new_df = []
        timestamps_resampled = np.arange(0., org_df.iloc[-1][' timestamp'], step=1. / FRAME_RATE)
        for col_name in org_df.columns:            
            # Get interpolation function
            f = interpolate.interp1d(x=org_df[' timestamp'], y=org_df[col_name], kind='linear')
            new_df.append( f(timestamps_resampled) )
        # This approach does not work properly
#         n_resampled_points = int(1 + (len(org_df[' timestamp']) - 1) * FRAME_RATE / csv_frame_rate)
#         for col_name in org_df.columns:            
#             new_col = scipy.signal.resample(np.array(org_df[col_name]), num=n_resampled_points)
#             new_df.append(new_col)
        new_df = pd.DataFrame(np.array(new_df).T, columns=org_df.columns) 
                
    # Add first and second derivatives of selected features
    diff_features = dict()
    for feature_name in diff_selected_features:
        diff_features['diff_' + feature_name] =  np.diff(new_df[feature_name], prepend=new_df[feature_name][0])
        diff_features['diff2_' + feature_name] = np.diff(diff_features['diff_' + feature_name], 
                                                         prepend=diff_features['diff_' + feature_name][0])    
    new_df = new_df.assign(**diff_features)
            
    # Annotate frames
    # Add annotation columns for nod and shake
    new_df = new_df.assign(nod=(1 if head_gesture == 'nod' else 0))
    new_df = new_df.assign(shake=(1 if head_gesture == 'shake' else 0))
#     print(new_df)
#     print(np.where(np.array(new_df.nod) == 1))
#     print(np.where(np.array(new_df.shake) == 1))
    
    # Save as new annotated dataframe
#     print(new_df)
    new_df.to_csv(f'{output_dir}/{sid}_{head_gesture}_{sessid}.csv', index=False)
    cnt += 1
    print(f'Time taken: {time.time() - start_time} s\n')        
#     break

print(f'\nGenerated {cnt} annotated feature files.')

Processing SESSID 1
	 head gesture: nod	 sessid_prefix: 2008.12.05.16.03.15_Operator	 sid: 1
	 original dataframe length 80
	csv frame rate: 49.968374446552815
	REsampling from 49.968374446552815 to 30.0
Time taken: 0.4323842525482178 s

Processing SESSID 1
	 head gesture: nod	 sessid_prefix: 2008.12.05.16.03.15_Operator	 sid: 1
	 original dataframe length 55
	csv frame rate: 50.0
	REsampling from 50.0 to 30.0
Time taken: 0.8656871318817139 s

Processing SESSID 1
	 head gesture: nod	 sessid_prefix: 2008.12.05.16.03.15_Operator	 sid: 1
	 original dataframe length 40
	csv frame rate: 50.0
	REsampling from 50.0 to 30.0
Time taken: 1.2725677490234375 s

Processing SESSID 1
	 head gesture: nod	 sessid_prefix: 2008.12.05.16.03.15_Operator	 sid: 1
	 original dataframe length 148
	csv frame rate: 49.98299897993879
	REsampling from 49.98299897993879 to 30.0
Time taken: 1.7885921001434326 s

Processing SESSID 1
	 head gesture: nod	 sessid_prefix: 2008.12.05.16.03.15_Operator	 sid: 1
	 original d

Time taken: 16.470151662826538 s

Processing SESSID 1
	 head gesture: nod	 sessid_prefix: 2008.12.14.14.47.07_Operator	 sid: 1
	 original dataframe length 65
	csv frame rate: 49.960967993754885
	REsampling from 49.960967993754885 to 30.0
Time taken: 16.897766590118408 s

Processing SESSID 1
	 head gesture: nod	 sessid_prefix: 2008.12.14.14.47.07_Operator	 sid: 1
	 original dataframe length 28
	csv frame rate: 50.0
	REsampling from 50.0 to 30.0
Time taken: 17.29212784767151 s

Processing SESSID 1
	 head gesture: nod	 sessid_prefix: 2008.12.14.14.47.07_Operator	 sid: 1
	 original dataframe length 60
	csv frame rate: 50.0
	REsampling from 50.0 to 30.0
Time taken: 17.709778308868408 s

Processing SESSID 1
	 head gesture: nod	 sessid_prefix: 2008.12.14.14.47.07_Operator	 sid: 1
	 original dataframe length 50
	csv frame rate: 50.0
	REsampling from 50.0 to 30.0
Time taken: 18.119755029678345 s

Processing SESSID 1
	 head gesture: nod	 sessid_prefix: 2008.12.14.14.47.07_Operator	 sid: 1
	 orig

Time taken: 33.25810503959656 s

Processing SESSID 5
	 head gesture: nod	 sessid_prefix: 2009.01.28.15.35.20_Operator	 sid: 5
	 original dataframe length 85
	csv frame rate: 49.97025580011898
	REsampling from 49.97025580011898 to 30.0
Time taken: 33.708914041519165 s

Processing SESSID 5
	 head gesture: nod	 sessid_prefix: 2009.01.28.15.35.20_Operator	 sid: 5
	 original dataframe length 42
	csv frame rate: 50.0
	REsampling from 50.0 to 30.0
Time taken: 34.1169707775116 s

Processing SESSID 5
	 head gesture: nod	 sessid_prefix: 2009.01.28.15.35.20_Operator	 sid: 5
	 original dataframe length 52
	csv frame rate: 50.0
	REsampling from 50.0 to 30.0
Time taken: 34.526339530944824 s

Processing SESSID 5
	 head gesture: nod	 sessid_prefix: 2009.01.28.15.35.20_Operator	 sid: 5
	 original dataframe length 68
	csv frame rate: 49.96271439224459
	REsampling from 49.96271439224459 to 30.0
Time taken: 34.95295834541321 s

Processing SESSID 5
	 head gesture: nod	 sessid_prefix: 2009.01.28.15.35.20_Op

Time taken: 49.33380651473999 s

Processing SESSID 6
	 head gesture: nod	 sessid_prefix: 2009.05.22.15.17.45_User	 sid: 6
	 original dataframe length 27
	csv frame rate: 50.0
	REsampling from 50.0 to 30.0
Time taken: 49.726773262023926 s

Processing SESSID 7
	 head gesture: nod	 sessid_prefix: 2009.05.25.11.23.09_User	 sid: 7
	 original dataframe length 73
	csv frame rate: 49.96530187369883
	REsampling from 49.96530187369883 to 30.0
Time taken: 50.16223073005676 s

Processing SESSID 7
	 head gesture: nod	 sessid_prefix: 2009.05.25.11.23.09_User	 sid: 7
	 original dataframe length 70
	csv frame rate: 49.9637943519189
	REsampling from 49.9637943519189 to 30.0
Time taken: 50.58591270446777 s

Processing SESSID 7
	 head gesture: nod	 sessid_prefix: 2009.05.25.11.23.09_User	 sid: 7
	 original dataframe length 60
	csv frame rate: 50.0
	REsampling from 50.0 to 30.0
Time taken: 51.00170683860779 s

Processing SESSID 7
	 head gesture: nod	 sessid_prefix: 2009.05.25.11.23.09_User	 sid: 7
	 origi

Time taken: 65.6841492652893 s

Processing SESSID 8
	 head gesture: nod	 sessid_prefix: 2009.05.26.10.19.53_User	 sid: 8
	 original dataframe length 40
	csv frame rate: 50.0
	REsampling from 50.0 to 30.0
Time taken: 66.0804750919342 s

Processing SESSID 8
	 head gesture: nod	 sessid_prefix: 2009.05.26.10.19.53_User	 sid: 8
	 original dataframe length 68
	csv frame rate: 49.96271439224459
	REsampling from 49.96271439224459 to 30.0
Time taken: 66.49959707260132 s

Processing SESSID 8
	 head gesture: nod	 sessid_prefix: 2009.05.26.10.19.53_User	 sid: 8
	 original dataframe length 105
	csv frame rate: 49.975973089860645
	REsampling from 49.975973089860645 to 30.0
Time taken: 67.00618481636047 s

Processing SESSID 8
	 head gesture: nod	 sessid_prefix: 2009.05.26.10.19.53_User	 sid: 8
	 original dataframe length 55
	csv frame rate: 50.0
	REsampling from 50.0 to 30.0
Time taken: 67.4229416847229 s

Processing SESSID 9
	 head gesture: other	 sessid_prefix: 2008	 sid: 9
	 original dataframe len

Time taken: 87.97019863128662 s

Processing SESSID 2
	 head gesture: shake	 sessid_prefix: 2008.12.05.16.03.15_User	 sid: 2
	 original dataframe length 45
	csv frame rate: 50.0
	REsampling from 50.0 to 30.0
Time taken: 88.37626242637634 s

Processing SESSID 2
	 head gesture: shake	 sessid_prefix: 2008.12.05.16.03.15_User	 sid: 2
	 original dataframe length 35
	csv frame rate: 49.99999999999999
	REsampling from 49.99999999999999 to 30.0
Time taken: 88.76876449584961 s

Processing SESSID 2
	 head gesture: shake	 sessid_prefix: 2008.12.05.16.03.15_User	 sid: 2
	 original dataframe length 110
	csv frame rate: 49.97707473635947
	REsampling from 49.97707473635947 to 30.0
Time taken: 89.23560738563538 s

Processing SESSID 2
	 head gesture: shake	 sessid_prefix: 2008.12.05.16.03.15_User	 sid: 2
	 original dataframe length 73
	csv frame rate: 50.0
	REsampling from 50.0 to 30.0
Time taken: 89.66095185279846 s

Processing SESSID 1
	 head gesture: shake	 sessid_prefix: 2008.12.14.14.47.07_Operator

Time taken: 103.73471260070801 s

Processing SESSID 1
	 head gesture: shake	 sessid_prefix: 2009.01.06.12.41.42_Operator	 sid: 1
	 original dataframe length 63
	csv frame rate: 49.95970991136181
	REsampling from 49.95970991136181 to 30.0
Time taken: 104.16227388381958 s

Processing SESSID 1
	 head gesture: shake	 sessid_prefix: 2009.01.06.12.41.42_Operator	 sid: 1
	 original dataframe length 91
	csv frame rate: 49.972237645752365
	REsampling from 49.972237645752365 to 30.0
Time taken: 104.60356903076172 s

Processing SESSID 1
	 head gesture: shake	 sessid_prefix: 2009.01.06.12.41.42_Operator	 sid: 1
	 original dataframe length 85
	csv frame rate: 49.97025580011898
	REsampling from 49.97025580011898 to 30.0
Time taken: 105.04131603240967 s

Processing SESSID 1
	 head gesture: shake	 sessid_prefix: 2009.01.06.14.53.49_Operator	 sid: 1
	 original dataframe length 32
	csv frame rate: 50.0
	REsampling from 50.0 to 30.0
Time taken: 105.43723106384277 s

Processing SESSID 1
	 head gesture: sh

Time taken: 119.27105832099915 s

Processing SESSID 6
	 head gesture: shake	 sessid_prefix: 2009.05.22.15.17.45_User	 sid: 6
	 original dataframe length 37
	csv frame rate: 50.0
	REsampling from 50.0 to 30.0
Time taken: 119.66743111610413 s

Processing SESSID 6
	 head gesture: shake	 sessid_prefix: 2009.05.22.15.17.45_User	 sid: 6
	 original dataframe length 56
	csv frame rate: 49.99999999999999
	REsampling from 49.99999999999999 to 30.0
Time taken: 120.09976005554199 s

Processing SESSID 6
	 head gesture: shake	 sessid_prefix: 2009.05.22.15.17.45_User	 sid: 6
	 original dataframe length 32
	csv frame rate: 50.0
	REsampling from 50.0 to 30.0
Time taken: 120.49245476722717 s

Processing SESSID 6
	 head gesture: shake	 sessid_prefix: 2009.05.22.15.17.45_User	 sid: 6
	 original dataframe length 70
	csv frame rate: 49.9637943519189
	REsampling from 49.9637943519189 to 30.0
Time taken: 120.91745638847351 s

Processing SESSID 6
	 head gesture: shake	 sessid_prefix: 2009.05.22.15.17.45_User	 